# Etiquetamento Autônomo de Perguntas do StackOverflow com Deep Learning

- Bruno Pilão
- Maria Mello
- Larissa Nobrega
- Fernanda Moyses 


O objetivo do trabalho é desenvolver um modelo Deep Learning para o etiquetamento autônomo (*multi-label classification*) de perguntas do StackOverflow.

CONJUNTO DE DADOS

A base de dados a ser utilizada é a StackLite, disponível em https://github.com/dgrtwo/StackLite.

A StackLite é uma versão simplificada e pré-processada de uma parte dos dados do Stack Overflow, contendo perguntas e suas tags associadas.

Embora o repositório possa conter outros arquivos, o foco será nos dados que permitem mapear perguntas (corpo e/ou título) para suas tags.

Obs.: Uma única pergunta pode ter múltiplas tags, o que a torna um problema de classificação multilabel. Pense em uma forma de priorizar uma das tags para simplificar o problema.



In [67]:
# Importando Bibliotecas

import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
import os
import seaborn as sns
import matplotlib.pyplot as plt
import time

## Análise breve dataset 


### Dataset Auxiliar 

Este dataset constitui um dicionário que mapeia cada questão gerada no Stack Overflow às suas respectivas categorias temáticas. O conjunto de dados auxiliar será utilizado para realizar uma análise prévia com o objetivo de avaliar e classificar os tipos de respostas associadas a cada pergunta.

In [62]:
#Importação dataset tags
df_tags = pd.read_csv('data\question_tags.csv\question_tags.csv.gz',compression='gzip')
df_tags.head()

,Id,Tag
0,1,data
1,4,c#
2,4,winforms
3,4,type-conversion
4,4,decimal


In [63]:
#Validandop quantidade de itens na datable
len(df_tags)

52224835

In [64]:
#Validando tipos de tag
print("Quantidade de itens por ID")
for i in range(10):
    print(i+1,df_tags[df_tags['Id'] == i+1]['Tag'].values)

Quantidade de itens por ID
1 ['data']
2 []
3 []
4 ['c#' 'winforms' 'type-conversion' 'decimal' 'opacity']
5 []
6 ['html' 'css' 'css3' 'internet-explorer-7']
7 []
8 ['c#' 'code-generation' 'j#' 'visualj#']
9 ['c#' '.net' 'datetime']
10 []


Observamos que as tags não possuem um ordem sequencial e podem conter um ou multiplos valores dentro de um ID.

In [65]:

# GroupBy simples: ID → lista de tags
grouped_mini = df_tags.groupby('Id')['Tag'].apply(list).reset_index()

print(f"✅ Grouped criado: {len(grouped_mini):,} IDs únicos")

# Salvar em CSV ()
grouped_mini.to_csv('stackoverflow_grouped_tags.csv', index=False)
print("✅ Backup CSV: stackoverflow_grouped_tags.csv")

✅ Grouped criado: 17,763,486 IDs únicos
✅ Backup CSV: stackoverflow_grouped_tags.csv


In [66]:
# Comprimir CSV (reduz ~80%)
df = pd.read_csv('stackoverflow_grouped_tags.csv')
df.to_csv('stackoverflow_grouped_tags.csv.gz', compression='gzip', index=False)

Com isso avaliamos que este dataset serve como um "Dicionário" para identificar o tipo de pergunta que o usuario realizou no stackoverflow, sendo que uma tg pode contem um ou múltiplos assuntos envolvidos.


## Dataset Principal

O dataset principal consiste em perguntas técnicas geradas por usuários da plataforma Stack Overflow. Este conjunto de dados contém questões formuladas por desenvolvedores e profissionais de tecnologia que buscam soluções para problemas específicos em suas áreas de atuação.

In [ ]:
#Importação dataset questions
df_questions = pd.read_csv('data\question_tags.csv\questions.csv.gz',compression='gzip')
df_questions.head()